<a href="https://colab.research.google.com/github/LucasG99/modelo-fraude/blob/main/C%C3%B3pia_de_Ponderada_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Descrição da ponderada:**

Objetivo: Otimizar um modelo de rede neural pré-treinado para detecção de fraudes em cartões de crédito. Aplicar técnicas avançadas de ajuste fino de hiperparâmetros, como grid search e random search, com o objetivo de aprimorar as métricas de desempenho do modelo, incluindo precisão, recall, F1-score e AUC-ROC. A atividade também exige uma comparação entre o modelo otimizado e o modelo original, permitindo avaliar o impacto das modificações nos hiperparâmetros sobre o desempenho geral.

In [1]:
%pip install gdown
import gdown

In [2]:
arquivo_destino_colab = "dataset.csv"
doc_id = "1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl"
URL = f"https://drive.google.com/uc?id={doc_id}"
gdown.download(URL, arquivo_destino_colab, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl
From (redirected): https://drive.google.com/uc?id=1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl&confirm=t&uuid=9659871f-afc7-4732-800d-18b5406cd8ac
To: /content/dataset.csv
100%|██████████| 151M/151M [00:02<00:00, 68.3MB/s]


'dataset.csv'

Daqui pra frente é o pré processamento ...

In [4]:
import pandas as pd

# Carregar o dataset
df = pd.read_csv("dataset.csv")

# Verificar dimensões
print(df.shape)

# Mostrar primeiras linhas
print(df.head())

# Conferir informações das colunas
print(df.info())

(284807, 31)
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V

In [5]:
df['Class'].value_counts(normalize=True)


,proportion
Class,
0,0.998273
1,0.001727


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
import tensorflow as tf
from tensorflow import keras

# 1. Features e alvo
X = df.drop("Class", axis=1)
y = df["Class"]

# 2. Escalonamento
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. Divisão treino/teste estratificada
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, stratify=y, random_state=42
)

# 4. Modelo simples (baseline)
model = keras.Sequential([
    keras.layers.Dense(32, activation="relu", input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# 5. Treino com class weights
# Calcula pesos inversamente proporcionais
neg, pos = np.bincount(y_train)
total = neg + pos
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)
class_weights = {0: weight_for_0, 1: weight_for_1}

history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=2048,
    validation_split=0.2,
    class_weight=class_weights,
    verbose=2
)

# 6. Avaliação no teste
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred, digits=4))

auc = roc_auc_score(y_test, y_pred_probs)
print("AUC-ROC:", auc)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
78/78 - 2s - 22ms/step - accuracy: 0.3135 - loss: 0.5876 - val_accuracy: 0.6485 - val_loss: 0.6692
Epoch 2/10
78/78 - 1s - 14ms/step - accuracy: 0.7834 - loss: 0.3308 - val_accuracy: 0.9500 - val_loss: 0.3789
Epoch 3/10
78/78 - 1s - 9ms/step - accuracy: 0.9183 - loss: 0.2382 - val_accuracy: 0.9756 - val_loss: 0.2622
Epoch 4/10
78/78 - 1s - 12ms/step - accuracy: 0.9539 - loss: 0.1992 - val_accuracy: 0.9834 - val_loss: 0.1923
Epoch 5/10
78/78 - 1s - 8ms/step - accuracy: 0.9676 - loss: 0.1673 - val_accuracy: 0.9838 - val_loss: 0.1605
Epoch 6/10
78/78 - 1s - 8ms/step - accuracy: 0.9702 - loss: 0.1587 - val_accuracy: 0.9824 - val_loss: 0.1450
Epoch 7/10
78/78 - 1s - 8ms/step - accuracy: 0.9707 - loss: 0.1477 - val_accuracy: 0.9814 - val_loss: 0.1311
Epoch 8/10
78/78 - 0s - 5ms/step - accuracy: 0.9700 - loss: 0.1411 - val_accuracy: 0.9810 - val_loss: 0.1208
Epoch 9/10
78/78 - 1s - 8ms/step - accuracy: 0.9724 - loss: 0.1384 - val_accuracy: 0.9827 - val_loss: 0.1101
Epoch 10/10
78/7

In [8]:
!pip install scikeras

In [12]:
from sklearn.metrics import classification_report, roc_auc_score

def train_and_evaluate(n_units=32, n_hidden=1, dropout=0.0, lr=1e-3, epochs=10, batch_size=1024):
    model = keras.Sequential()
    model.add(keras.Input(shape=(X_train.shape[1],)))
    for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_units, activation="relu"))
        if dropout > 0:
            model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(1, activation="sigmoid"))
    opt = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])
    model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2,
        class_weight=class_weights,
        verbose=0
    )
    y_proba = model.predict(X_test, verbose=0)
    y_pred = (y_proba >= 0.5).astype(int)
    report = classification_report(y_test, y_pred, output_dict=True)
    auc = roc_auc_score(y_test, y_proba)
    return {
        "n_units": n_units,
        "n_hidden": n_hidden,
        "dropout": dropout,
        "lr": lr,
        "epochs": epochs,
        "batch_size": batch_size,
        "precision_1": report["1"]["precision"],
        "recall_1": report["1"]["recall"],
        "f1_1": report["1"]["f1-score"],
        "auc": auc
    }

results = []
results.append(train_and_evaluate(n_units=32, n_hidden=1, dropout=0.0, lr=1e-3, epochs=10))
results.append(train_and_evaluate(n_units=64, n_hidden=2, dropout=0.2, lr=1e-3, epochs=20))
results.append(train_and_evaluate(n_units=128, n_hidden=2, dropout=0.3, lr=5e-4, epochs=20))
results.append(train_and_evaluate(n_units=256, n_hidden=3, dropout=0.4, lr=1e-4, epochs=30))

pd.DataFrame(results)


,n_units,n_hidden,dropout,lr,epochs,batch_size,precision_1,recall_1,f1_1,auc
0,32,1,0.0,0.0010,10,1024,0.159596,0.533784,0.245723,0.855728
1,64,2,0.2,0.0010,20,1024,0.001711,0.986486,0.003416,0.490601
2,128,2,0.3,0.0005,20,1024,0.001722,0.993243,0.003439,0.497193
3,256,3,0.4,0.0001,30,1024,0.001721,0.993243,0.003437,0.467482


# Relatório da Atividade – Detecção de Fraudes

A base tem quase **285 mil transações**, mas só **0,17% são fraudes**. Isso já mostrou que a acurácia não é suficiente, porque um modelo que só prevê “não fraude” já teria 99% de acerto.
Por isso, decidi focar em **precision, recall, F1 e AUC-ROC**.

* Escalei os dados com `StandardScaler`.
* Dividi em treino e teste de forma estratificada.
* Usei **class weights** para dar mais importância às fraudes.

Modelo inicial (baseline)

Treinei uma rede simples: 1 camada com 32 neurônios, dropout de 0.3 e saída sigmoide.
Resultados (classe fraude):

* **Precision = 0,076**
* **Recall = 0,865**
* **F1 = 0,140**
* **AUC = 0,967**

Ou seja: o modelo acerta quase todas as fraudes (recall alto), mas gera muitos falsos positivos (precision baixa).

Tentativas de otimização

Tentei primeiro usar `RandomizedSearchCV` com SciKeras, mas deu erros de versão no Colab (scores `nan`). Desisti dessa parte e resolvi fazer uma **busca manual simples**, mudando hiperparâmetros:

* número de camadas ocultas (1 a 3)
* número de neurônios (32 a 256)
* dropout (0 a 0.4)
* taxa de aprendizado (1e-3 até 1e-4)
* épocas (10 a 30)

Rodei quatro combinações principais.

Resultados dos experimentos

| Modelo (neurônios/camadas) | Precision₁ | Recall₁ | F1₁        | AUC   |
| -------------------------- | ---------- | ------- | ---------- | ----- |
| 32 / 1 / dropout 0.0       | **0,1596** | 0,5338  | **0,2457** | 0,856 |
| 64 / 2 / dropout 0.2       | 0,0017     | 0,9865  | 0,0034     | 0,491 |
| 128 / 2 / dropout 0.3      | 0,0017     | 0,9932  | 0,0034     | 0,497 |
| 256 / 3 / dropout 0.4      | 0,0017     | 0,9932  | 0,0034     | 0,467 |

Comparação com o baseline

* **Baseline:** Recall muito alto (0,865) e AUC ótimo (0,967), mas precisão muito baixa (0,076).
* **Melhor teste:** Rede pequena (32 neurônios, 1 camada, sem dropout). Teve precisão maior (0,16) e F1 melhor (0,25), mas perdeu recall e AUC.
* **Redes mais complexas:** Ficaram praticamente inúteis (AUC \~0,49 e F1 \~0).

Conclusão

* O **baseline** é bom quando a prioridade é **não deixar fraudes passarem** (recall alto, AUC alto).
* O **modelo mais simples** deu um equilíbrio melhor (subiu precisão e F1), mas sacrificou recall.
* Modelos maiores não ajudaram, provavelmente por causa do desbalanceamento extremo.

No geral, percebi que **a rede menor foi a mais estável** e que ajustes simples já mudam bastante a precisão e recall. Para trabalhos futuros, daria pra testar **mudança do threshold de decisão** ou técnicas como **SMOTE** para tentar aumentar a precisão sem perder recall.